In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import hashlib
from datetime import datetime

# ============ DCGAN Architecture (Same as before) ============
class Generator(nn.Module):
    def __init__(self, nz=100, ngf=64, nc=3):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)


class Discriminator(nn.Module):
    def __init__(self, nc=3, ndf=64):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x).view(-1, 1).squeeze(1)


# ============ Enhanced Chaotic Map ============
class EnhancedChaoticKeyGenerator:
    def __init__(self, initial_seed=0.123456789, r=3.99):
        self.x = initial_seed
        self.r = r

    def logistic_map(self, x):
        """Logistic map: x_{n+1} = r * x_n * (1 - x_n)"""
        return (self.r * x * (1 - x)) % 1.0

    def tent_map(self, x):
        """Tent map for additional chaos"""
        return 2 * x if x < 0.5 else 2 * (1 - x)

    def generate_sequence(self, length):
        """Generate chaotic sequence with higher complexity"""
        sequence = []
        for _ in range(length):
            # Alternate between maps for more chaos
            self.x = self.logistic_map(self.x)
            self.x = self.tent_map(self.x)
            sequence.append(self.x)
        return np.array(sequence)

    def generate_key_sequences(self, num_sequences=8, seq_length=512):
        """Generate multiple independent chaotic key sequences"""
        keys = []
        for i in range(num_sequences):
            # Different initial conditions for each sequence
            self.x = (self.x + 0.123 * (i + 1)) % 1.0
            key_seq = self.generate_sequence(seq_length)
            keys.append(key_seq)
        return keys


# ============ Enhanced DCGAN Key Generator ============
class EnhancedDCGANKeyGenerator:
    def __init__(self, nz=100, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.nz = nz
        self.generator = Generator(nz=nz).to(device)
        self.chaotic_gen = EnhancedChaoticKeyGenerator()

    def load_pretrained(self, path):
        """Load pre-trained DCGAN generator"""
        self.generator.load_state_dict(torch.load(path, map_location=self.device))
        self.generator.eval()

    def generate_key_image(self, noise=None):
        """Generate synthetic image using DCGAN"""
        if noise is None:
            noise = torch.randn(1, self.nz, 1, 1, device=self.device)

        with torch.no_grad():
            fake_image = self.generator(noise)

        return fake_image

    def extract_base_key(self, image_tensor, timestamp=None, nonce=None, message_hash=None):
        """
        Enhanced key extraction with message hash integration
        """
        img_np = image_tensor.cpu().numpy().squeeze()
        pixel_sum = np.sum(img_np, axis=(0, 1, 2))

        if timestamp is None:
            timestamp = datetime.now().timestamp()

        if nonce is None:
            nonce = np.random.randint(0, 2**32)

        # Integrate message hash for message-specific keys
        if message_hash is None:
            message_hash = ""

        # Combine all components
        key_data = f"{pixel_sum}_{timestamp}_{nonce}_{message_hash}".encode()

        # Generate SHA-512 hash
        base_key = hashlib.sha512(key_data).digest()

        return base_key, timestamp, nonce

    def generate_encryption_keys(self, seq_length=512, message=None):
        """
        Complete key generation pipeline with message awareness
        """
        # Generate message hash if provided
        message_hash = None
        if message is not None:
            if isinstance(message, str):
                message_hash = hashlib.sha256(message.encode()).hexdigest()[:16]
            elif isinstance(message, torch.Tensor):
                message_hash = hashlib.sha256(message.cpu().numpy().tobytes()).hexdigest()[:16]

        # Generate DCGAN image
        noise = torch.randn(1, self.nz, 1, 1, device=self.device)
        key_image = self.generate_key_image(noise)

        # Extract base key with message hash
        base_key, timestamp, nonce = self.extract_base_key(key_image, message_hash=message_hash)

        # Initialize chaotic generator with base key
        seed = int.from_bytes(base_key[:8], byteorder='big') / (2**64)
        self.chaotic_gen.x = seed

        # Generate 8 chaotic key sequences
        key_sequences = self.chaotic_gen.generate_key_sequences(
            num_sequences=8,
            seq_length=seq_length
        )

        # Convert to tensors for easier use
        key_tensors = [torch.FloatTensor(ks).to(self.device) for ks in key_sequences]

        return {
            'base_key': base_key,
            'key_sequences': key_sequences,
            'key_tensors': key_tensors,
            'timestamp': timestamp,
            'nonce': nonce,
            'key_image': key_image,
            'noise': noise,
            'message_hash': message_hash
        }


# ============ String Preprocessing Utilities ============
class StringPreprocessor:
    """Convert strings to tensor format and back"""

    def __init__(self, max_length=512):
        self.max_length = max_length
        self.pad_token = 0
        self.start_token = 1
        self.end_token = 2

    def string_to_tensor(self, text):
        """Convert string to integer tensor (ASCII values)"""
        # Convert to ASCII values
        ascii_vals = [ord(c) for c in text]

        # Add start and end tokens
        tokens = [self.start_token] + ascii_vals + [self.end_token]

        # Pad or truncate
        if len(tokens) < self.max_length:
            tokens = tokens + [self.pad_token] * (self.max_length - len(tokens))
        else:
            tokens = tokens[:self.max_length-1] + [self.end_token]

        return torch.LongTensor(tokens)

    def tensor_to_string(self, tensor):
        """Convert integer tensor back to string"""
        # Remove padding and special tokens
        tokens = tensor.cpu().numpy()

        # Find end token
        try:
            end_idx = np.where(tokens == self.end_token)[0][0]
            tokens = tokens[1:end_idx]  # Remove start and end tokens
        except:
            tokens = tokens[1:]  # Just remove start token

        # Remove padding
        tokens = tokens[tokens != self.pad_token]

        # Convert back to characters
        try:
            text = ''.join([chr(int(t)) for t in tokens if 0 < t < 128])
        except:
            text = ""

        return text

    def batch_strings_to_tensor(self, strings):
        """Convert batch of strings to tensor"""
        return torch.stack([self.string_to_tensor(s) for s in strings])

    def batch_tensor_to_strings(self, tensor):
        """Convert batch of tensors to strings"""
        return [self.tensor_to_string(t) for t in tensor]


# ============ Usage Example ============
if __name__ == "__main__":
    print("="*70)
    print("ENHANCED DCGAN KEY GENERATOR - STRING SUPPORT")
    print("="*70)

    # Initialize
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    key_gen = EnhancedDCGANKeyGenerator(device=device)
    preprocessor = StringPreprocessor(max_length=256)

    # Test with string
    test_message = "Hello, this is a secret message!"
    print(f"\nOriginal message: '{test_message}'")

    # Convert to tensor
    message_tensor = preprocessor.string_to_tensor(test_message)
    print(f"Tensor shape: {message_tensor.shape}")

    # Generate keys for this message
    keys = key_gen.generate_encryption_keys(
        seq_length=256,
        message=test_message
    )

    print(f"\nKey generation complete:")
    print(f"  Base key size: {len(keys['base_key'])} bytes")
    print(f"  Number of key sequences: {len(keys['key_sequences'])}")
    print(f"  Each sequence length: {len(keys['key_sequences'][0])}")
    print(f"  Message hash: {keys['message_hash']}")
    print(f"  Key space: 2^440 combinations")

    # Test back conversion
    recovered = preprocessor.tensor_to_string(message_tensor)
    print(f"\nRecovered message: '{recovered}'")
    print(f"Match: {recovered == test_message}")

    print("\n" + "="*70)

ENHANCED DCGAN KEY GENERATOR - STRING SUPPORT

Original message: 'Hello, this is a secret message!'
Tensor shape: torch.Size([256])

Key generation complete:
  Base key size: 64 bytes
  Number of key sequences: 8
  Each sequence length: 256
  Message hash: b17bc84328a21850
  Key space: 2^440 combinations

Recovered message: 'Hello, this is a secret message!'
Match: True

